# FWI in Azure project

## Create Experimentation Docker image

FWI demo based on: 
This project ports devito (https://github.com/opesci/devito) into Azure and runs tutorial notebooks at:
https://nbviewer.jupyter.org/github/opesci/devito/blob/master/examples/seismic/tutorials/



In this notebook we create a custom docker image that will be used to run the devito demo notebooks in AzureML. 

 - We transparently create a docker file, a conda environment .yml file, build the docker image and push it into dockerhub. Azure ACR could also be used for storing docker images. 
 - The conda environment .yml file lists conda and pip installs, and separates all python dependencies from the docker installs. 
 - The dockerfile is generic. The only AzureML depedency is azureml-sdk pip installable package in conda environment .yml file
 - The created docer image will be run in following notebook in a container on the local AzureVM or on a remote AzureML compute cluster. This AzureML pattern decouples experimentation (or training) job definition (experimentation script, data location, dependencies and docker image) happening on the control plane machine that runs this notebook, from the elastically allocated and Azure managed VM/cluster that does the actual training/experimentation computation.
 
<a id='user_input_requiring_steps'></a>
User input requiring steps:
 - [Fill in and save docker image name settings, if needed. ](#docker_image_settings)
 - [Update DOCKER_CONTAINER_MOUNT_POINT to match our local path](#docker_image_settings)
 - [Set docker build and test flags](#docker_build_test_settings) 


In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import sys, os
import shutil
import urllib

import platform
import math
import docker

In [3]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1060-azure-x86_64-with-debian-10.0'

'/workspace/examples/imaging/azureml_devito/notebooks'

<a id='docker_build_test_settings'></a>
#### Setup docker image build and test process. 
 - devito tests take abou 15 mins (981.41 seconds). When running this notebook for first time make:
     > docker_build_no_cache = '--no-cache'  
     > docker_test_run_devito_tests = True
     
[Back](#user_input_requiring_steps) to summary of user input requiring steps.

In [4]:
docker_build_no_cache = '--no-cache'  # '--no-cache' # or '' #
docker_test_run_devito_tests = True # True # False

##### Import utilities functions

In [5]:
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
        
auxiliary_files_dir = os.path.join(*(['.', 'src']))
paths_to_append = [os.path.join(os.getcwd(), auxiliary_files_dir)]
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import project_utils
prj_consts = project_utils.project_consts()

[None]

##### Create experimentation docker file

In [6]:
dotenv_file_path = os.path.join(*(prj_consts.DOTENV_FILE_PATH))
dotenv_file_path

'./../not_shared/general.env'

In [7]:
!pwd

/workspace/examples/imaging/azureml_devito/notebooks


<a id='docker_image_settings'></a>

##### Input here docker image settings 
in cell below we use [dotenv](https://github.com/theskumar/python-dotenv) to overwrite docker image properties already save in dotenv_file_path. Change as needed, e.g. update azureml_sdk version if using a different version.

[Back](#user_input_requiring_steps) to summary of user input requiring steps.

In [8]:
# SDK changes often, so we'll keep its version transparent 
import dotenv

# EXPERIMENTATION_IMAGE_VERSION should:
# - match sdk version in fwi01_conda_env01 environmnet in conda_env_fwi01_azureml_sdk.v1.0.XX.yml file below
# -  match the conda env yml file name, e.g. conda_env_fwi01_azureml_sdk.v1.0.62.yml referenced in 
#      Dockerfile_fwi01_azureml_sdk.v1.0.62
dotenv.set_key(dotenv_file_path, 'EXPERIMENTATION_IMAGE_VERSION', 'sdk.v1.0.65')
dotenv.set_key(dotenv_file_path, 'EXPERIMENTATION_IMAGE_TAG', 'fwi01_azureml')
dotenv.set_key(dotenv_file_path, 'DOCKER_CONTAINER_MOUNT_POINT', '/datadrive01/prj/DeepSeismic/examples/imaging/azureml_devito/notebooks')

(True, 'EXPERIMENTATION_IMAGE_VERSION', 'sdk.v1.0.65')

(True, 'EXPERIMENTATION_IMAGE_TAG', 'fwi01_azureml')

(True,
 'DOCKER_CONTAINER_MOUNT_POINT',
 '/datadrive01/prj/DeepSeismic/examples/imaging/azureml_devito/notebooks')

In [9]:
%load_ext dotenv
%dotenv $dotenv_file_path

docker_file_location = os.path.join(*(prj_consts.AML_EXPERIMENT_DIR + ['docker_build']))

docker_file_name = 'Dockerfile'+ '_' + os.getenv('EXPERIMENTATION_IMAGE_TAG')
conda_dependency_file_name = 'conda_env'+ '_' + os.getenv('EXPERIMENTATION_IMAGE_TAG')
devito_conda_dependency_file_name = 'devito_conda_env'+'.yml'
docker_image_name = os.getenv('DOCKER_LOGIN') + '/' + os.getenv('EXPERIMENTATION_IMAGE_TAG')
image_version = os.getenv('EXPERIMENTATION_IMAGE_VERSION')
if image_version!="":
    docker_file_name = docker_file_name +'_'+ image_version
    conda_dependency_file_name = conda_dependency_file_name+'_'+ image_version
    docker_image_name = docker_image_name +':'+ image_version
conda_dependency_file_name=conda_dependency_file_name+'.yml'

docker_file_dir = os.path.join(*([os.getcwd(), docker_file_location]))
os.makedirs(docker_file_dir, exist_ok=True)
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
conda_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file_name]))

docker_image_name
conda_dependency_file_name
conda_file_path
docker_file_dir
docker_file_path

'georgedockeraccount/fwi01_azureml:sdk.v1.0.65'

'conda_env_fwi01_azureml_sdk.v1.0.65.yml'

'/workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build/conda_env_fwi01_azureml_sdk.v1.0.65.yml'

'/workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build'

'/workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build/Dockerfile_fwi01_azureml_sdk.v1.0.65'

In [10]:
%%writefile $conda_file_path
name: fwi01_conda_env01
    
#https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow_gpu-1.13.1-cp37-cp37m-linux_x86_64.whl    
# https://github.com/dask/dask-tutorial

channels:
  - anaconda
  - conda-forge
dependencies:
  - python=3.6 # 3.6 req by tf, not 3.7.2 
  - dask
  - distributed
  - h5py
  - matplotlib
  - nb_conda
  - notebook 
  - numpy 
  - pandas
  - pip
  - py-cpuinfo # all required by devito or dask-tutorial
  - pytables
  - python-graphviz
  - requests>=2.19.1
  - pillow
  - scipy
  - snakeviz
  - scikit-image
  - toolz
  - pip:
    - anytree # required by devito
    - azureml-sdk[notebooks,automl]==1.0.65
    - codepy # required by devito
    - papermill[azure]
    - pyrevolve # required by devito

Overwriting /workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build/conda_env_fwi01_azureml_sdk.v1.0.65.yml


In [11]:
%%writefile $docker_file_path 

FROM continuumio/miniconda3:4.7.10    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 \
    curl \
    git make \
    mpich \ 
    libmpich-dev && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ENV CONDA_ENV_FILE_NAME conda_env_fwi01_azureml_sdk.v1.0.65.yml
ADD $CONDA_ENV_FILE_NAME /tmp/$CONDA_ENV_FILE_NAME
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME fwi01_conda_env

RUN git clone https://github.com/opesci/devito.git  && \
    cd devito  && \
    /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f environment.yml && \
    pip install -e . 
    
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH   

RUN /opt/conda/bin/conda env update --name $CONDA_ENV_NAME -f /tmp/$CONDA_ENV_FILE_NAME && \
    /opt/conda/bin/conda clean  --yes --all

ENV PYTHONPATH=$PYTHONPATH:devito/app

# WORKDIR /devito     
    
CMD /bin/bash

Overwriting /workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build/Dockerfile_fwi01_azureml_sdk.v1.0.65


In [12]:
! ls -l $docker_file_dir

total 24
-rw-r--r-- 1 root root 1098 Sep 25 00:39 Dockerfile_fwi01_azureml_sdk.v1.0.60
-rw-r--r-- 1 root root 1098 Sep 26 19:04 Dockerfile_fwi01_azureml_sdk.v1.0.62
-rw-r--r-- 1 root root 1085 Oct  9 17:54 Dockerfile_fwi01_azureml_sdk.v1.0.65
-rw-r--r-- 1 root root  713 Sep 25 00:39 conda_env_fwi01_azureml_sdk.v1.0.60.yml
-rw-r--r-- 1 root root  713 Sep 26 19:04 conda_env_fwi01_azureml_sdk.v1.0.62.yml
-rw-r--r-- 1 root root  733 Oct  9 17:54 conda_env_fwi01_azureml_sdk.v1.0.65.yml


In [13]:
cli_command='docker build -t '+ docker_image_name + \
' -f ' + docker_file_path + \
' ' + docker_file_dir + ' ' +\
docker_build_no_cache  #'' #' --no-cache'


cli_command
! $cli_command

'docker build -t georgedockeraccount/fwi01_azureml:sdk.v1.0.65 -f /workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build/Dockerfile_fwi01_azureml_sdk.v1.0.65 /workspace/examples/imaging/azureml_devito/notebooks/./../temp/docker_build --no-cache'

Sending build context to Docker daemon  11.78kB
Step 1/15 : FROM continuumio/miniconda3:4.7.10
 ---> 4a51de2367be
Step 2/15 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 084695efc71b
Removing intermediate container 084695efc71b
 ---> 447e38875551
Step 3/15 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl     git make     mpich     libmpich-dev &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in d09c88390a5b
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:3 http://deb.debian.org/debian buster/main amd64 Packages [7899 kB]
Get:4 http://security.debian.org/debian-security buster/updates InRelease [39.1 kB]
Get:5 http://deb.debian.org/debian buster-updates/main amd64 Packages.diff/Index [1720 B]
Ign:5 http://deb.debian.org/debian buster-updates/main amd64 Packages.diff/Index
Get:6 http://de

Selecting previously unselected package libisl19:amd64.
Preparing to unpack .../04-libisl19_0.20-2_amd64.deb ...
Unpacking libisl19:amd64 (0.20-2) ...
Selecting previously unselected package libmpfr6:amd64.
Preparing to unpack .../05-libmpfr6_4.0.2-1_amd64.deb ...
Unpacking libmpfr6:amd64 (4.0.2-1) ...
Selecting previously unselected package libmpc3:amd64.
Preparing to unpack .../06-libmpc3_1.1.0-1_amd64.deb ...
Unpacking libmpc3:amd64 (1.1.0-1) ...
Selecting previously unselected package cpp-8.
Preparing to unpack .../07-cpp-8_8.3.0-6_amd64.deb ...
Unpacking cpp-8 (8.3.0-6) ...
Selecting previously unselected package cpp.
Preparing to unpack .../08-cpp_4%3a8.3.0-1_amd64.deb ...
Unpacking cpp (4:8.3.0-1) ...
Selecting previously unselected package libcurl4:amd64.
Preparing to unpack .../09-libcurl4_7.64.0-4_amd64.deb ...
Unpacking libcurl4:amd64 (7.64.0-4) ...
Selecting previously unselected package curl.
Preparing to unpack .../10-curl_7.64.0-4_amd64.deb ...
Unpacking curl (7.64.0-4) 

Setting up gfortran-8 (8.3.0-6) ...
Setting up gcc (4:8.3.0-1) ...
Setting up g++-8 (8.3.0-6) ...
Setting up gfortran (4:8.3.0-1) ...
update-alternatives: using /usr/bin/gfortran to provide /usr/bin/f95 (f95) in auto mode
update-alternatives: using /usr/bin/gfortran to provide /usr/bin/f77 (f77) in auto mode
Setting up g++ (4:8.3.0-1) ...
update-alternatives: using /usr/bin/g++ to provide /usr/bin/c++ (c++) in auto mode
Setting up libmpich-dev:amd64 (3.3-3) ...
update-alternatives: using /usr/include/x86_64-linux-gnu/mpich to provide /usr/include/x86_64-linux-gnu/mpi (mpi-x86_64-linux-gnu) in auto mode
Processing triggers for libc-bin (2.28-10) ...
Removing intermediate container d09c88390a5b
 ---> 768917e3839d
Step 4/15 : ENV CONDA_ENV_FILE_NAME conda_env_fwi01_azureml_sdk.v1.0.65.yml
 ---> Running in f82224761f1f
Removing intermediate container f82224761f1f
 ---> 6fc45a3f7f12
Step 5/15 : ADD $CONDA_ENV_FILE_NAME /tmp/$CONDA_ENV_FILE_NAME
 ---> 956a64671412
Step 6/15 : ENV CONDA_DIR /

  Stored in directory: /root/.cache/pip/wheels/90/7e/74/bb640d77775e6b6a78bcc3120f9fea4d2a28b2706de1cff37d
  Stored in directory: /root/.cache/pip/wheels/58/80/1c/4b18cf63778aafa8f62c37395448164667143bbbb20bf16b9d
  Stored in directory: /root/.cache/pip/wheels/f4/53/e7/b53cf7ba45381b676bbd5eaaedc19ae82e1c397e9c1766ddf4
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
  Stored in directory: /root/.cache/pip/wheels/63/9d/8e/37c3f6506ed3f152733a699e92d8e0c9f5e5f01dea262f80ad
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Stored in directory: /root/.cache/pip/wheels/e1/8b/65/3294e5b727440250bda09e8c0153b7ba19d328f661605cb151
  Stored in directory: /root/.cache/pip/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
  Stored in directory: /root/.cache/p

pandas-0.25.1        | 11.4 MB   | ########## | 100% 
icu-58.2             | 22.5 MB   | ########## | 100% 
dask-2.5.2           | 12 KB     | ########## | 100% 
ca-certificates-2019 | 132 KB    | ########## | 100% 
libedit-3.1.20181209 | 188 KB    | ########## | 100% 
numpy-1.17.2         | 4 KB      | ########## | 100% 
yaml-0.1.7           | 84 KB     | ########## | 100% 
certifi-2019.9.11    | 154 KB    | ########## | 100% 
imageio-2.6.0        | 3.3 MB    | ########## | 100% 
pexpect-4.7.0        | 82 KB     | ########## | 100% 
asn1crypto-1.0.1     | 161 KB    | ########## | 100% 
hdf5-1.10.4          | 5.3 MB    | ########## | 100% 
prometheus_client-0. | 42 KB     | ########## | 100% 
packaging-19.2       | 30 KB     | ########## | 100% 
appdirs-1.4.3        | 16 KB     | ########## | 100% 
cycler-0.10.0        | 13 KB     | ########## | 100% 
cython-0.29.13       | 2.2 MB    | ########## | 100% 
libstdcxx-ng-9.1.0   | 4.0 MB    | ########## | 100% 
cached-property-1.5. | 11 KB

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
click-7.0-py37_0.conda                       120 KB
pyflakes-2.1.1-py37_0.conda                  106 KB
tblib-1.4.0-py_0.tar.bz2                      14 KB
libsodium-1.0.16-h1bed415_0.conda            214 KB
kiwisolver-1.1.0-py37he6710b0_0.conda         82 KB
jinja2-2.10.3-py_0.tar.bz2                    95 KB
zict-1.0.0-py_0.tar.bz2                       12 KB
ipython-7.8.0-py37h39e3cac_0.conda           985 KB
traitlets-4.3.3-py37_0.tar.bz2               138 KB
libstdcxx-ng-9.1.0-hdf63c60_0.conda          3.1 MB
pycparser-2.19-py37_0.conda                  171 KB
cgen-2019.1-py_0.tar.bz2                      16 KB
imagesize-1.1.0-py37_0.conda                   9 KB
jedi-0.15.1-py37_0.conda                     704 KB
zlib-1.2.11-h7b6447c_3.conda                 103 KB
importlib_metadata-0.23-py37_0.tar.bz2        43 KB
nb_conda_kernels-2.2.2-py37_0.conda           39 KB
heapdict-1.0.1-

nbconvert-5.6.0-py36_1.tar.bz2               494 KB

---------------------------------------------------
Total:                                     926.1 MB

Removed click-7.0-py37_0.conda
Removed pyflakes-2.1.1-py37_0.conda
Removed tblib-1.4.0-py_0.tar.bz2
Removed libsodium-1.0.16-h1bed415_0.conda
Removed kiwisolver-1.1.0-py37he6710b0_0.conda
Removed jinja2-2.10.3-py_0.tar.bz2
Removed zict-1.0.0-py_0.tar.bz2
Removed ipython-7.8.0-py37h39e3cac_0.conda
Removed traitlets-4.3.3-py37_0.tar.bz2
Removed libstdcxx-ng-9.1.0-hdf63c60_0.conda
Removed pycparser-2.19-py37_0.conda
Removed cgen-2019.1-py_0.tar.bz2
Removed imagesize-1.1.0-py37_0.conda
Removed jedi-0.15.1-py37_0.conda
Removed zlib-1.2.11-h7b6447c_3.conda
Removed importlib_metadata-0.23-py37_0.tar.bz2
Removed nb_conda_kernels-2.2.2-py37_0.conda
Removed heapdict-1.0.1-py_0.conda
Removed cytoolz-0.10.0-py37h7b6447c_0.conda
Removed babel-2.7.0-py_0.tar.bz2
Removed libxml2-2.9.9-hea5a465_1.conda
Removed mpc-1.1.0-h10f8cd9_1.conda
Removed m

Removed certifi-2019.9.11-py36_0.tar.bz2
Removed gstreamer-1.14.0-hb453b48_1.tar.bz2
Removed pandoc-2.2.3.2-0.tar.bz2
Removed pandas-0.25.1-py36he6710b0_0.tar.bz2
Removed chardet-3.0.4-py36_1003.tar.bz2
Removed libedit-3.1.20181209-hc058e9b_0.tar.bz2
Removed cached-property-1.5.1-py36_0.tar.bz2
Removed pyqt-5.9.2-py36h22d08a2_1.tar.bz2
Removed tornado-6.0.3-py36h7b6447c_0.tar.bz2
Removed psutil-5.6.3-py36h7b6447c_0.tar.bz2
Removed libsodium-1.0.16-h1bed415_0.tar.bz2
Removed gmp-6.1.2-hb3b607b_0.tar.bz2
Removed pexpect-4.7.0-py36_0.tar.bz2
Removed nbformat-4.4.0-py36_0.tar.bz2
Removed python-3.6.9-h265db76_0.tar.bz2
Removed docutils-0.15.2-py36_0.tar.bz2
Removed coverage-4.5.4-py36h7b6447c_0.tar.bz2
Removed xz-5.2.4-h14c3975_4.tar.bz2
Removed mccabe-0.6.1-py36_1.tar.bz2
Removed jsonschema-3.0.2-py36_0.tar.bz2
Removed fontconfig-2.13.0-h9420a91_0.tar.bz2
Removed networkx-2.3-py_0.tar.bz2
Removed importlib_metadata-0.23-py36_0.tar.bz2
Removed send2trash-1.5.0-py36_0.tar.bz2
Removed pluggy

Docker containers can be run using python docker sdk

In [14]:
docker_image_name

sh_command='bash -c "pwd;python -c \'import azureml.core;print(azureml.core.VERSION)\'"'
sh_command
client = docker.from_env()
client.containers.run(docker_image_name, 
                      remove=True,
                      volumes={os.getenv('DOCKER_CONTAINER_MOUNT_POINT'): {'bind': '/workspace', 'mode': 'rw'}},
                      working_dir='/',
                      command=sh_command)

'georgedockeraccount/fwi01_azureml:sdk.v1.0.65'

'bash -c "pwd;python -c \'import azureml.core;print(azureml.core.VERSION)\'"'

b'/\n1.0.65\n'

Docker containers can also be run in cli 

Here we also create a log file to capture commands execution in container. If flag docker_test_run_devito_tests is True, we run 
and capture test commands output. Tests take abou 15 minutes to run. If flag docker_test_run_devito_tests is False, we show the results of a previous session. 

In [15]:
fwi01_log_file = os.path.join(*(['.', 'fwi01_azureml_buildexperimentationdockerimage.log']))
fwi01_log_file

'./fwi01_azureml_buildexperimentationdockerimage.log'

#### Create command for running devito tests, capture output in a log file, save log file outside container

In [16]:
if docker_test_run_devito_tests:
    run_devito_tests_command = ' python -m pytest tests/ '   + \
'> ' + fwi01_log_file +' 2>&1; ' + \
' mv ' + fwi01_log_file + ' /workspace/'  
    
    with open(os.path.join(*(['.', 'fwi01_azureml_buildexperimentationdockerimage.log'])), "w") as crt_log_file:
        print('Before running e13n container... ', file=crt_log_file)
    print('\ncontent of devito tests log file before testing:')
    !cat $fwi01_log_file
else:
    run_devito_tests_command =  '' 

# run_devito_tests_command =  'ls -l > ./fwi01_azureml_buildexperimentationdockerimage.log 2>&1;  mv ./fwi01_azureml_buildexperimentationdockerimage.log /workspace/'
run_devito_tests_command


content of devito tests log file before testing:
Before running e13n container... 


' python -m pytest tests/ > ./fwi01_azureml_buildexperimentationdockerimage.log 2>&1;  mv ./fwi01_azureml_buildexperimentationdockerimage.log /workspace/'

In [17]:
cli_command='docker run -it --rm  --name fwi01_azureml_container ' +\
' -v '+os.getenv('DOCKER_CONTAINER_MOUNT_POINT')+':/workspace:rw ' + \
docker_image_name + \
' /bin/bash -c "conda env list ; ls -l /devito/tests;  '  + \
'python -c \'import azureml.core;print(azureml.core.VERSION)\'; '  + \
'cd /devito; '  + \
run_devito_tests_command +\
' "'

cli_command
! $cli_command
# # ============= 774 passed, 70 skipped, 1 xfailed in 1106.76 seconds =============
print('\ncontent of devito tests log file after testing:')
!cat $fwi01_log_file

'docker run -it --rm  --name fwi01_azureml_container  -v /datadrive01/prj/DeepSeismic/examples/imaging/azureml_devito/notebooks:/workspace:rw georgedockeraccount/fwi01_azureml:sdk.v1.0.65 /bin/bash -c "conda env list ; ls -l /devito/tests;  python -c \'import azureml.core;print(azureml.core.VERSION)\'; cd /devito;  python -m pytest tests/ > ./fwi01_azureml_buildexperimentationdockerimage.log 2>&1;  mv ./fwi01_azureml_buildexperimentationdockerimage.log /workspace/ "'

# conda environments:
#
base                     /opt/conda
fwi01_conda_env       *  /opt/conda/envs/fwi01_conda_env

total 508
-rw-r--r-- 1 root root 11521 Oct  9 17:55 conftest.py
-rw-r--r-- 1 root root  6425 Oct  9 17:55 test_adjoint.py
-rw-r--r-- 1 root root 13882 Oct  9 17:55 test_autotuner.py
-rw-r--r-- 1 root root  9727 Oct  9 17:55 test_checkpointing.py
-rw-r--r-- 1 root root  1095 Oct  9 17:55 test_constant.py
-rw-r--r-- 1 root root 53290 Oct  9 17:55 test_data.py
-rw-r--r-- 1 root root   481 Oct  9 17:55 test_dependency_bugs.py
-rw-r--r-- 1 root root 16585 Oct  9 17:55 test_derivatives.py
-rw-r--r-- 1 root root 30846 Oct  9 17:55 test_dimension.py
-rw-r--r-- 1 root root 21233 Oct  9 17:55 test_dle.py
-rw-r--r-- 1 root root  1157 Oct  9 17:55 test_docstrings.py
-rw-r--r-- 1 root root 26251 Oct  9 17:55 test_dse.py
-rw-r--r-- 1 root root  8612 Oct  9 17:55 test_gradient.py
-rw-r--r-- 1 root root 15229 Oct  9 17:55 test_interpolation.py
-rw-r--r-- 1 root root 31514 Oct  9 17:55 

In [18]:
docker_pwd = os.getenv('DOCKER_PWD')
docker_login = os.getenv('DOCKER_LOGIN')
!docker login -u=$docker_login -p=$docker_pwd

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [19]:
# %%bash
!docker push {docker_image_name}

The push refers to repository [docker.io/georgedockeraccount/fwi01_azureml]

9886dd1f: Preparing 
99799fa2: Preparing 
ec869ef0: Preparing 
55141871: Preparing 
f8fc4c9a: Preparing 
ba47210e: Preparing 


99799fa2: Pushing  1.134GB/3.024GBPushing  552.1MB/3.024GBPushing  823.1MB/3.024GBPushing  861.2MB/2.81GBPushing  978.4MB/3.024GB

99799fa2: Pushing  2.206GB/3.024GBPushing  1.825GB/2.81GBPushing  1.896GB/2.81GB

9886dd1f: Pushed   3.099GB/3.024GBPushing  2.445GB/2.81GBPushing  2.798GB/3.024GBsdk.v1.0.65: digest: sha256:3639f4b469e0fb59c8531022448f240bff49d9d06c8d9e788656d5dd1bfbb07b size: 1800


In [20]:
# !jupyter nbconvert 010_CreateExperimentationDockerImage_GeophysicsTutorial_FWI_Azure_devito --to html
print('Finished running 010_CreateExperimentationDockerImage_GeophysicsTutorial_FWI_Azure_devito!')

Finished running 010_CreateExperimentationDockerImage_GeophysicsTutorial_FWI_Azure_devito!
